# Assignment Netwerkanalyse

## Notebook made by

**Gebruik graag dit formaat**

* Voor de namen:  voornaam rest van je naam, voornaam rest van je naam,....
* je studentnummers: hetzelfde: scheidt met `,`
* je emails: hetzelfde: scheidt met `,`
* voor je groep: alleen de hoofdletter (dus voor de groep van Marx zou je `A` kiezen)

__Namen__:Anoniem
__Emails__:Anoniem
__Student id__:Anoniem
__Groep__:Anoniem

## Toelichting

* De meeste opgaven worden automatisch nagekeken. Bij vrijwel alle opdrachten staan er een paar tests onder de opdracht, dit is voornamelijk om te zorgen dat je de juiste type output geeft. Dit zijn dus *NIET* alle tests, die komen er bij het graden nog bij.
* Elke vraag is 1 punt waard, tenzij anders aangegeven. Soms is die punt onderverdeeld in deelpunten, maar niet altijd. 

## Voor het inleveren!

* Pas niet de cellen aan, vooral niet die je niet kunt editen. Dit levert problemen op bij nakijken. Twijfel je of je per ongeluk iets hebt gewijzigd, kopieer dan bij inleveren je antwoorden naar een nieuw bestand, zodat het niet fout kan gaan.

* Zorg dat de code goed runt van boven naar beneden, verifieer dat door boven in Kernel -> Restart & Run All uit te voeren

## Na het inleveren!

* Het gebeurt erg vaak dat mensen een "leeg bestand" inleveren. Vaak een andere versie van de opgave die nog ergens op je computer rondslingerde. Zonde van al je werk toch!
* Dus, lever **minstens een half uur voor tijd in**. Download dan wat je hebt ingeleverd op Canvas. Geef het een andere naam om verwarring te voorkomen. En draai alle cellen, en bekijk het. Geen syntax fouten? Alle vragen gemaakt? Dan zit het vast wel goed, en hoef je niet in de zenuwen te zitten.

# Week 3
* Alle vragen zijn één punt waard

In [ ]:
%matplotlib inline
import networkx as nx
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from nose.tools import assert_equal, assert_count_equal

# Deel 1: First Course Chap 2

* Maak vragen 1-40

### Ook met programmeren

* Check je antwoorden op vragen 8-11 door het gerichte netwerk te programmeren, en alle vragen ook te programmeren in Networkx.
* Zet de netwerken in Fig 2-13 en 2.14 in Networkx, probeer ze ook zo mooi uit te printen (zie assignment week 2), en check je antwoorden op de vragen erover.
* Als je het leuk vindt kan je dit ook met de andere voorbeelden doen. Je weet dan wel zeker dat je het goed hebt.
    * Op het tentamen mag je ook programmeren.....


# Deel 2: Easley & Kleinberg Chapter 13

De volgende vragen hebben betrekking op [hoofdstuk 13 van Easley and Kleinberg](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch13.pdf). Hieronder staat de networkx graaf gebaseerd op figuur 13.8, pas deze vooral aan om te zorgen dat je functies altijd werken.

In [ ]:
EK = nx.DiGraph()
EK.add_edges_from([(6, 2), (6, 7), (7, 8), (11, 7), (1, 8), (12, 13), (8, 13),
                   (18, 13), (15, 18), (15, 14), (13, 14), (14, 9), (9, 3),
                   (3, 8), (4, 1), (4, 5), (5, 10), (16, 10), (17, 16), (15,
                                                                         16),
                   (9, 4), (9, 15)])

nx.draw_circular(EK, with_labels=True)

## Vraag 5

* Maak een functie `find_SCC` die gegeven een gerichte graaf een set teruggeeft met daarin alle nodes die behoren tot de grootste SCC (strongly connected component).

* Maak een functie `find_IN` die gegeven een gerichte graaf een set teruggeeft met daarin alle nodes die behoren tot de IN (dat zijn alle knopen met een pad naar de SCC, maar vanuit de SCC kan je niet naar die knopen komen).

* Maak een functie `find_OUT` die gegeven een gerichte graaf een set teruggeeft met daarin alle nodes die behoren tot de OUT (ja, dit snap je nu zelf wel hoop ik).

In [ ]:
def find_SCC(G):
    scc = list(nx.strongly_connected_components(G))
    scc_max = max(scc, key=len)
    return set(scc_max)


def find_OUT(G):
    scc = find_SCC(G)
    return {node for node in G.nodes if node not in scc and any(s not in scc for s in G.successors(node))}

def find_IN(G):
    scc = find_SCC(G)
    find_inset = {k for n in scc for k in G.nodes if k not in scc and k not in G.neighbors(n)}
    return find_inset

# even testen op figuur 13.8 uit het boek
find_SCC(EK), find_IN(EK), find_OUT(EK)

In [ ]:
assert_equal(type(find_SCC(EK)), set)
assert_equal(type(find_OUT(EK)), set)
assert_equal(type(find_IN(EK)), set)
assert find_SCC(EK).issubset(EK.nodes())
assert find_IN(EK).issubset(EK.nodes())
assert find_OUT(EK).issubset(EK.nodes())

In [ ]:
# resultaat graaf tekenen
def draw_web(G):
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True)
    color = sns.color_palette("hls", 7)
    #tubes = find_tubes(G)
    legenda = [
        'SCC', 'IN', 'OUT', ] #'Tendrils (IN)', 'Tendrils (OUT)', 'Tubes','Disconnected' ]
    listofnodes = [
        find_SCC(G),
        find_IN(G),
        find_OUT(G),
        #find_tendrils_IN(G) - tubes,
        #find_tendrils_OUT(G) - tubes, tubes,
        #find_disconnected(G)
    ]
    for n, (label, nodes, c) in enumerate(zip(legenda, listofnodes, color)):
        nx.draw_networkx_nodes(
            G, pos, nodelist=nodes, node_color=[c], label=label)
    nx.draw_networkx_edges(G, pos)
    plt.legend()
    plt.axis('off')
    plt.show()


draw_web(EK)

# Deel 3: Clustering coëfficiënt

In [ ]:
G = nx.karate_club_graph()

nx.draw(G, with_labels=True)

## Vraag 3.1
Maak een functie `clusteringcoefficient(G)` die een graaf accepteert als input, en voor alle knopen de clusteringcoëfficiënt ([Sectie 3.1 E&amp;K](http://www.cs.cornell.edu/home/kleinber/networks-book/networks-book-ch03.pdf)) berekent.
                
<strong>Bereken dit zelf, gebruik geen NetworkX functie die dit al doet zoals `clustering` en `average_clustering`. </strong> Je mag natuurlijk wel NetworkX handigheidjes als `G.neighbors(v)` gebruiken.

Als output krijg je een dict  met de knopen van `G` als sleutels en de clustering coeffiecient van die knoop als bijbehorende waarde.   Geef waarde 0 als een knoop degree 1 heeft.


De verwachte output voor deze graaf is 1 voor knoop 7 en 2/3 voor knoop 3 en 0 voor knoop 11. etc. 
  

In [ ]:
def clusteringcoefficient(G):
    clustering_coefficients = {}
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        k = len(neighbors)
        if k == 0 or k == 1:
            clustering_coefficients[node] = 0.0
        else:
            num_of_link_between_neighbor = 0
            
            for i in range(k):
                for j in range(i+1, k):
                    if G.has_edge(neighbors[i], neighbors[j]):
                        num_of_link_between_neighbor += 1
                        
            clustering_coefficients[node] = 2.0 * num_of_link_between_neighbor / (k * (k-1))
            
    return clustering_coefficients




# even er een mooi plotje van maken
pd.Series( clusteringcoefficient(G) ).sort_values().plot(kind='barh',figsize=(8,8))


clusteringcoefficient(G)

In [ ]:
assert isinstance(clusteringcoefficient(G), dict)
for n in clusteringcoefficient(G):
    assert n in G
    assert isinstance(clusteringcoefficient(G)[n],float)

## Vraag 3.1.1

Maak de functie `cc_nice(G, v)` nu nog een keer maar gebruik nu alleen maar `G.has_edge()` van networkx.
Chack dat hij echt hetzelfde doet als je eerdere functie en als networkx clustering.

In [ ]:
def cc_nice(G, v):
    numberOfNeighbors = 0
    neighbors = []
    for i in G.nodes:
        if G.has_edge(i, v):
            numberOfNeighbors +=1
            neighbors.append(i)
    
    if numberOfNeighbors == 1 or numberOfNeighbors == 0:
        return 0.0
    
    else:
        num_of_link_between_neighbor = 0
        for i in neighbors:
            for j in G.nodes:
                 if G.has_edge(i, j) and j is not v:
                    for k in G.nodes:
                        if k == v and G.has_edge(j , k):
                            num_of_link_between_neighbor +=1
        return num_of_link_between_neighbor / (numberOfNeighbors * (numberOfNeighbors-1))


cc_nice(G, 5)
# check dat de twee functies hetzelfde resultaat geven voor elke knoop op de karateclub




# check dat de twee functies hetzelfde resultaat geven voor elke knoop op de karateclub
all(clusteringcoefficient(G)[n] == cc_nice(G,n) for n in G)

## Vraag 3.1.2

Maak de functie `cc_nicer(G, v)` nu nog een keer maar gebruik nu alleen maar  `G.subgraph` en `nx.density` van networkx.
Check dat hij echt hetzelfde doet als je eerdere functie en als networkx clustering.

In [ ]:
def cc_nice(G, v):
    numberOfNeighbors = 0
    neighbors = []
    for i in G.nodes:
        if G.has_edge(i, v):
            numberOfNeighbors +=1
            neighbors.append(i)
    
    if numberOfNeighbors == 1 or numberOfNeighbors == 0:
        return 0.0
    
    else:
        num_of_link_between_neighbor = 0
        for i in neighbors:
            for j in G.nodes:
                 if G.has_edge(i, j) and j is not v:
                    for k in G.nodes:
                        if k == v and G.has_edge(j , k):
                            num_of_link_between_neighbor +=1
        return num_of_link_between_neighbor / (numberOfNeighbors * (numberOfNeighbors-1))


cc_nice(G, 5)
# check dat de twee functies hetzelfde resultaat geven voor elke knoop op de karateclub




# check dat de twee functies hetzelfde resultaat geven voor elke knoop op de karateclub
all(clusteringcoefficient(G)[n] == cc_nice(G,n) for n in G)

## Vraag 3.2

2. Maak de functie  `is_complete(G)`  die `True` teruggeeft precies dan als het ongerichte netwerk  `G` een volledig netwerk is en `False` als dat niet het geval is.  Gebruik **alleen** de networkx functies `nx.number_of_nodes(G)` en `G.number_of_edges()`.

In [ ]:
def is_complete(G):
    N = nx.number_of_nodes(G)
    max_edges = N * (N-1) // 2  
    return G.number_of_edges() == max_edges
is_complete(G)

In [ ]:
assert isinstance(is_complete(G), bool)